In [1]:
#All Libraries used in the code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [7]:
#Reading the data from the csv files
hour3 = pd.read_csv('1_6pm - 9pm.csv')
hour6 = pd.read_csv('2_9pm - 12am.csv')
hour9 = pd.read_csv('3_12am - 3am.csv')
hour12 = pd.read_csv('4_3am - 6am.csv')
hour15 = pd.read_csv('5_6am - 9am.csv')
hour18 = pd.read_csv('6_9am - 12pm.csv')
hour21 = pd.read_csv('7_12pm - 3pm.csv')
hour24 = pd.read_csv('8_3pm - 6pm.csv')